In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# Creating Dataframes

In [ ]:
folder = 'E:/Desktop/Uber kmeans workshop/Dataset/'

In [ ]:
apr14 = pd.read_csv(folder+'uber-raw-data-apr14.csv')
may14 = pd.read_csv(folder+'uber-raw-data-may14.csv')
jun14 = pd.read_csv(folder+'uber-raw-data-jun14.csv')
jul14 = pd.read_csv(folder+'uber-raw-data-jul14.csv')
aug14 = pd.read_csv(folder+'uber-raw-data-aug14.csv')
sep14 = pd.read_csv(folder+'uber-raw-data-sep14.csv')

In [ ]:
df = pd.concat([apr14,may14,jun14,jul14,aug14,sep14])

In [ ]:
df

In [ ]:
##Now we will do data pre-processing on "Date/Time" feature

In [ ]:
#new = pd.to_datetime(apr14['Date/Time'])
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

In [ ]:
df

# Histogram

In [ ]:
df['Date/Time'].dt.time

In [ ]:
df['Time'] = df['Date/Time'].dt.time.apply(lambda x : int(x.strftime('%H%M%S')))

In [ ]:
df

In [ ]:
plt.hist(df['Time'])

In [ ]:
sns.histplot(df['Time'])
#we could see that the maximum no. of rides take place between 3pm. to 9pm.

# Now, lets divide our dataset based on morning and evening rides

In [ ]:
morning_df_index = (df['Time'] > 5000) & (df['Time'] < 110000)
morning_df = df[morning_df_index]
print(morning_df)

In [ ]:
evening_df_index = (df['Time'] > 150000) & (df['Time'] < 220000)
evening_df = df[evening_df_index]
print(evening_df)

In [ ]:
#Now, we will get coordinates of cab booked in morning and evening
#For this, we will combine random 10000 Latitude and longitude in one single array
morning_coords = morning_df[['Lat','Lon']].sample(10000,random_state=10).values
print(morning_coords)

In [ ]:
evening_coords = evening_df[['Lat','Lon']].sample(10000,random_state=10).values
print(evening_coords)

# Plotting morning and evening rides on map

In [ ]:
morning_map = folium.Map([40.7709, -73.949],zoom_start = 11,tiles = 'stamen toner')
for coords in morning_coords:
    folium.CircleMarker(location = coords,fill=True,radius=1).add_to(morning_map)
morning_map

In [ ]:
evening_map = folium.Map([40.6975,-73.9967],zoom_start = 11,tiles = 'stamen toner')
for coords in evening_coords:
    folium.CircleMarker(location = coords,fill=True,radius=1,color='red').add_to(evening_map)
evening_map

# Now, lets find clusters(hotspot areas) in morning rides, evening rides and whole dataset 


# 1) Morning ride clusters

In [ ]:
clusters = 6   #no. of cluster should be decided by using elbow method. I did it below for whole dataset
model = KMeans(n_clusters = clusters,init = 'random',max_iter=300)

In [ ]:
model.fit(morning_df[['Lat','Lon']])

In [ ]:
morning_centroids = model.cluster_centers_
morning_centroids

In [ ]:
for i,coords in enumerate(morning_centroids):
    folium.Marker(location = coords,popup='centroid {}'.format(i+1)).add_to(morning_map)
morning_map

# 2) Evening ride clusters

In [ ]:
#For evening rides
clusters = 6
model = KMeans(n_clusters = clusters,init='random',max_iter=300)

In [ ]:
model.fit(evening_df[['Lat','Lon']])

In [ ]:
evening_centroids = model.cluster_centers_
evening_centroids

In [ ]:
for i,coords in enumerate(evening_centroids):
    folium.Marker(location = coords,popup='centroid {}'.format(i+1)).add_to(evening_map)
evening_map

# 3) Clusters for whole dataset 
#        Elbow method 

In [ ]:
#for whole dataset
#elbow method using WCSS
wcss = []
for i in range(1,11):
    model = KMeans(n_clusters = i)
    model.fit(df[['Lat','Lon']])
    w = model.inertia_       #WCSS value
    wcss.append(w)
    print(wcss[i-1])

In [ ]:
plt.plot(wcss)
#We could see that 8 could be the no. of clusters. So, take 8 as nclusters.

In [ ]:
cluster = 8
model = KMeans(n_clusters = cluster)
model.fit(df[['Lat','Lon']])
model.inertia_

In [ ]:
centroids = model.cluster_centers_
centroids

In [ ]:
map = folium.Map(location=[40.79658011772687, -73.87341741832425], zoom_start = 10, tiles='Stamen Toner')
for i, coordinate in enumerate(centroids):
    folium.Marker(coordinate, popup='Centroid {}'.format(i+1)).add_to(map)
map

# Let's Predict for the new ride.

In [ ]:
new_ride = (40.70647056912189, -73.91116590442799)      #suppose new ride is coming from this location
folium.Marker(new_ride, popup='New Rider', icon=folium.Icon(color='green')).add_to(map)
map

In [ ]:
#now, lets predict which hotspot's cab will be assigned to this new ride
centroid_idx = model.predict([new_ride])
centroid_idx

In [ ]:
centroids[centroid_idx]

In [ ]:
folium.Marker(centroids[centroid_idx][0], icon=folium.Icon(color='red')).add_to(map)
map